In [ ]:
!pip install datasets
!pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
dataset = load_dataset("TruongSinhAI/deepcad_prompt_json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['uid', 'input', 'output', 'instruction'],
        num_rows: 156783
    })
    validation: Dataset({
        features: ['uid', 'input', 'output', 'instruction'],
        num_rows: 8814
    })
    test: Dataset({
        features: ['uid', 'input', 'output', 'instruction'],
        num_rows: 7924
    })
})

In [ ]:
# Merge train, validation, and test splits into a single dataset
merged_dataset = dataset['train'].to_pandas()
merged_dataset = pd.concat([merged_dataset, dataset['validation'].to_pandas()])
merged_dataset = pd.concat([merged_dataset, dataset['test'].to_pandas()])

# Calculate the split size (same length for each)
split_size = len(merged_dataset) // 3

# Create new train, validation, and test datasets
new_train_df = merged_dataset.iloc[:split_size]
new_val_df = merged_dataset.iloc[split_size:2*split_size]
new_test_df = merged_dataset.iloc[2*split_size:]

#Convert back to datasets
new_train_ds = Dataset.from_pandas(new_train_df)
new_val_ds = Dataset.from_pandas(new_val_df)
new_test_ds = Dataset.from_pandas(new_test_df)

new_dataset = DatasetDict({
    'part_1': new_train_ds,
    'part_2': new_val_ds,
    'part_3': new_test_ds
})

new_dataset


DatasetDict({
    part_1: Dataset({
        features: ['uid', 'input', 'output', 'instruction', '__index_level_0__'],
        num_rows: 57840
    })
    part_2: Dataset({
        features: ['uid', 'input', 'output', 'instruction', '__index_level_0__'],
        num_rows: 57840
    })
    part_3: Dataset({
        features: ['uid', 'input', 'output', 'instruction', '__index_level_0__'],
        num_rows: 57841
    })
})

In [ ]:
def Prompt_template(query, src_language, trg_language):
    instruction = f"Translate the following sentences from {src_language} to {trg_language}."
    prompt = (
        'Below is an instruction that describes a task, paired with an input that provides further context. '
        'Write a response that appropriately completes the request.\n'
        f'### Instruction:\n{instruction}\n'
        f'### Input:\n{query}\n### Response:'
    )
    return prompt


In [ ]:
''' LESS GPU =))'''
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="QuantFactory/LLaMAX3-8B-Alpaca-GGUF",
	filename="LLaMAX3-8B-Alpaca.Q8_0.gguf",
  n_gpu_layers=50,
  n_ctx=2048*4#18569


)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--QuantFactory--LLaMAX3-8B-Alpaca-GGUF/snapshots/22ec9c02263a9b6411ad3d850d0464f6efb6b314/./LLaMAX3-8B-Alpaca.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.at

In [ ]:
import json

def save_result(results, now):
  with open(f"translation_{now}.jsonl", "w", encoding="utf-8" ) as f:
      for item in results:
          f.write(json.dumps(item, ensure_ascii=False) + "\n")


In [36]:
results = []
now = 0

for data in new_dataset['part_1']:
  now+=1
  query = data['input']
  prompt = Prompt_template(
      query= query,
      src_language= "English",
      trg_language= "Vietnamese"
  )
  output = llm(prompt, max_tokens= 2048*4)

  data['input_vi'] = output['choices'][0]['text']

  results.append(data)

  if now%100==0:
    print(now, end = ' ')
    save_result(results, now)
  if now == 10:
    break

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1312.97 ms
llama_print_timings:      sample time =     543.39 ms /   263 runs   (    2.07 ms per token,   484.00 tokens per second)
llama_print_timings: prompt eval time =     695.54 ms /   331 tokens (    2.10 ms per token,   475.89 tokens per second)
llama_print_timings:        eval time =    9411.40 ms /   262 runs   (   35.92 ms per token,    27.84 tokens per second)
llama_print_timings:       total time =   10959.36 ms /   593 tokens
Llama.generate: prefix-match hit


KeyboardInterrupt: 

In [ ]:
# k = []
# for x in new_dataset.values():
#   for data in x:
#     k.append(len(data['input']))


**
5776   2  


In [32]:
# prompt = Prompt_template(
#     query= query,
#     src_language="English",
#     trg_language="Vietnamese"
# )

# # Gửi prompt đến mô hình (vì đây là mô hình kiểu Alpaca, dùng chế độ 'completion')
# output = llm(prompt, max_tokens=2048*4)  # hoặc dùng `llm.create_completion(prompt=prompt)` nếu bạn muốn thêm kiểm soát

# # In kết quả
# print(output['choices'][0]['text'])

In [33]:
# output['choices'][0]['text'].count('@@@')

In [34]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = AutoTokenizer.from_pretrained("LLaMAX/LLaMAX3-8B-Alpaca")
# model = AutoModelForCausalLM.from_pretrained("LLaMAX/LLaMAX3-8B-Alpaca").to(device)

In [35]:
# from transformers import AutoTokenizer, LlamaForCausalLM

# query = "你好，今天是个好日子"
# prompt = Prompt_template(query, 'Chinese', 'English')
# inputs = tokenizer(prompt, return_tensors="pt")

# generate_ids = model.generate(inputs.input_ids, max_length=2048)
# tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# # => "Hello, today is a good day"
